## hello_tm.py

In [1]:
__doc__="""
このプログラムでは、以下のようなデモを行い、Temporal Memoryに直接アクセスする方法を示しています。 TMインスタンスの作成方法、ベクトルを使った学習方法、予測値の取得方法、そして は状態を検査します。

このコードはシーケンス学習の非常にシンプルなバージョンを実行しています。 セルをカラムごとに表示します。TM は単純なシーケンス A->B->C->D->E で学習されます。
"""

from htm.bindings.sdr import SDR
from htm.algorithms import TemporalMemory as TM

In [2]:
# 特定の方法でSDRを印刷するためのユーティリティルーチンです
def formatBits(sdr):
  s = ''
  for c in range(sdr.size):
    if c > 0 and c % 10 == 0:
      s += ' '
    s += str(sdr.dense.flatten()[c])
  s += ' '
  return s

In [3]:
def printStateTM( tm ):
    # 内部状態をトレースするのに便利
    print("Active cells     " + formatBits(tm.getActiveCells()))
    print("Winner cells     " + formatBits(tm.getWinnerCells()))
    tm.activateDendrites(True)
    print("Predictive cells " + formatBits(tm.getPredictiveCells()))
    print("Anomaly", tm.anomaly * 100, "%")
    print("")


print("################################################################################")
print(__doc__)
print("################################################################################")
print("")
print("Creating the Temporal Memory")

################################################################################

このプログラムでは、以下のようなデモを行い、Temporal Memoryに直接アクセスする方法を示しています。 TMインスタンスの作成方法、ベクトルを使った学習方法、予測値の取得方法、そして は状態を検査します。

このコードはシーケンス学習の非常にシンプルなバージョンを実行しています。 セルをカラムごとに表示します。TM は単純なシーケンス A->B->C->D->E で学習されます。

################################################################################

Creating the Temporal Memory


In [4]:
tm = TM(columnDimensions = (50,),
        cellsPerColumn=1,
        initialPermanence=0.5,
        connectedPermanence=0.5,
        minThreshold=8,
        maxNewSynapseCount=20,
        permanenceIncrement=0.1,
        permanenceDecrement=0.0,
        activationThreshold=8,
        )
tm.printParameters()

Temporal Memory Parameters
version                   = 2
numColumns                = 50
cellsPerColumn            = 1
activationThreshold       = 8
initialPermanence         = 0.5
connectedPermanence       = 0.5
minThreshold              = 8
maxNewSynapseCount        = 20
permanenceIncrement       = 0.1
permanenceDecrement       = 0
predictedSegmentDecrement = 0
maxSegmentsPerCell        = 255
maxSynapsesPerSegment     = 255


In [5]:
print("""
テンポラリメモリに供給する入力を作成します。
各入力は、アクティブなミニカラムを表します。 
ここでは、5つのシーケンスを表すシンプルなSDRを作成します。
A -> B -> C -> D -> E  """)
dataset = { inp : SDR( tm.numberOfColumns() ) for inp in "ABCDE" }
dataset['A'].dense[0:10]  = 1   # Input SDR representing "A", corresponding to mini-columns 0-9
dataset['B'].dense[10:20] = 1   # Input SDR representing "B", corresponding to mini-columns 10-19
dataset['C'].dense[20:30] = 1   # Input SDR representing "C", corresponding to mini-columns 20-29
dataset['D'].dense[30:40] = 1   # Input SDR representing "D", corresponding to mini-columns 30-39
dataset['E'].dense[40:50] = 1   # Input SDR representing "E", corresponding to mini-columns 40-49


テンポラリメモリに供給する入力を作成します。
各入力は、アクティブなミニカラムを表します。 
ここでは、5つのシーケンスを表すシンプルなSDRを作成します。
A -> B -> C -> D -> E  


In [6]:
# SDR オブジェクトの高密度データをインプレースで更新したことを通知します
for z in dataset.values():
  z.dense = z.dense
for inp in "ABCDE":
  print("Input:", inp, " Bits:", formatBits( dataset[inp]) )
print("")

Input: A  Bits: 1111111111 0000000000 0000000000 0000000000 0000000000 
Input: B  Bits: 0000000000 1111111111 0000000000 0000000000 0000000000 
Input: C  Bits: 0000000000 0000000000 1111111111 0000000000 0000000000 
Input: D  Bits: 0000000000 0000000000 0000000000 1111111111 0000000000 
Input: E  Bits: 0000000000 0000000000 0000000000 0000000000 1111111111 



In [7]:
print("################################################################################")
print("")
print("""この簡単なシーケンスを学習用の一時記憶装置に送信します""")
print("""
計算方法は、学習および/または推論の1つのステップを実行します。注意: ここでは
我々は学習を行うだけですが、あなたは予測/推論と学習を行うことができます。
望むならば同じステップで（オンライン学習）.
""")
for inp in "ABCDE": # 各文字を順番に送る
  print("Input:", inp)
  activeColumns = dataset[inp]

  print(">>> tm.compute()")
  tm.compute(activeColumns, learn = True)

  printStateTM(tm)

print("""リセットコマンドは、シーケンスが終了したことを TM に伝えます。
はすべての状態をゼロにします。厳密には必要ではありませんが、ちょっとした
リセットなしではもっと難しく、TMはリセットした方が学習が早い.
""")
print(">>> tm.reset()")
print("")
tm.reset()

################################################################################

この簡単なシーケンスを学習用の一時記憶装置に送信します

計算方法は、学習および/または推論の1つのステップを実行します。注意: ここでは
我々は学習を行うだけですが、あなたは予測/推論と学習を行うことができます。
望むならば同じステップで（オンライン学習）.

Input: A
>>> tm.compute()
Active cells     1111111111 0000000000 0000000000 0000000000 0000000000 
Winner cells     1111111111 0000000000 0000000000 0000000000 0000000000 
Predictive cells 0000000000 0000000000 0000000000 0000000000 0000000000 
Anomaly 100.0 %

Input: B
>>> tm.compute()
Active cells     0000000000 1111111111 0000000000 0000000000 0000000000 
Winner cells     0000000000 1111111111 0000000000 0000000000 0000000000 
Predictive cells 0000000000 0000000000 0000000000 0000000000 0000000000 
Anomaly 100.0 %

Input: C
>>> tm.compute()
Active cells     0000000000 0000000000 1111111111 0000000000 0000000000 
Winner cells     0000000000 0000000000 1111111111 0000000000 0000000000 
Predictive cells 0000000000 0000000000 0000000000 0000000000 0000000000 
Anomaly 100.0 %



In [8]:
print("################################################################################")
print("")
print("""ベクトルの同じシーケンスを送り、次のような予測を見てください。
テンポラリメモリを使用しています。
以下は、アクティブセル、予測セル、アクティブセグメントをプリントアウトした
勝者セルです。
注目すべきは、アクティブな状態が1であるミニカラムは，現在の入力パターンのSDRと，
予測された、次の期待パターンのSDRを表す。
""")
for inp in "ABCDE":
  print("Input:", inp)
  activeColumns = dataset[inp]

  print(">>> tm.compute()")
  tm.compute(activeColumns, learn = False)

  printStateTM(tm)

################################################################################

ベクトルの同じシーケンスを送り、次のような予測を見てください。
テンポラリメモリを使用しています。
以下は、アクティブセル、予測セル、アクティブセグメントをプリントアウトした
勝者セルです。
注目すべきは、アクティブな状態が1であるミニカラムは，現在の入力パターンのSDRと，
予測された、次の期待パターンのSDRを表す。

Input: A
>>> tm.compute()
Active cells     1111111111 0000000000 0000000000 0000000000 0000000000 
Winner cells     1111111111 0000000000 0000000000 0000000000 0000000000 
Predictive cells 0000000000 1111111111 0000000000 0000000000 0000000000 
Anomaly 100.0 %

Input: B
>>> tm.compute()
Active cells     0000000000 1111111111 0000000000 0000000000 0000000000 
Winner cells     0000000000 1111111111 0000000000 0000000000 0000000000 
Predictive cells 0000000000 0000000000 1111111111 0000000000 0000000000 
Anomaly 0.0 %

Input: C
>>> tm.compute()
Active cells     0000000000 0000000000 1111111111 0000000000 0000000000 
Winner cells     0000000000 0000000000 1111111111 0000000000 0000000000 
Predictive cells 0000000000 0000000000 0000000000 1111111111 